## Interactive map to query the data

In [3]:
import os
from dotenv import load_dotenv
import psycopg2
from shapely import wkb
import ipywidgets as widgets
import geopandas as gpd
import folium
from folium.plugins import MousePosition, Draw, MiniMap
from decimal import Decimal
from attributes_descriptions import flood_descriptions as flood_des, landslide_descriptions as ls_des

# Get info from .env file
load_dotenv('../backend/database/.env')
db_name = os.getenv('DB_NAME')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
if not all([db_name, user, password, host]):
    raise ValueError("Plz set environment variables in .env file")

# Add Jupyter Widgets
scale_dropdown = widgets.Dropdown(
    options=['regioni', 'province'],
    description='Scale:',
    layout=widgets.Layout(width='auto', height='30px')
)
disaster_dropdown = widgets.Dropdown(
    options=['flood', 'landslide'],
    description='Disaster:',
    layout=widgets.Layout(width='auto', height='30px')
)
type_dropdown = widgets.Dropdown(
    options=['surface area', 'population', 'families', 'buildings', 'local business', 'cultural heritage'],
    description='Type:',
    layout=widgets.Layout(width='auto', height='30px')
)
province_dropdown = widgets.Dropdown(
    options=['milan'],
    description='province:',
    layout=widgets.Layout(width='auto', height='30px')
)
button = widgets.Button(
    description="Search",
    layout=widgets.Layout(width='80%', height='auto')
)

grid = widgets.GridspecLayout(4, 2)
grid[0, 0] = scale_dropdown
grid[1, 0] = disaster_dropdown
grid[2, 0] = type_dropdown
grid[3, 0] = province_dropdown
grid[:, 1] = button

output = widgets.Output()
text = widgets.Label(
    value="",
)


# Search button event
@output.capture(clear_output=True)
def button_clicked(b):
    text.value = "The map is loading, please wait.."
    display(text)
    query = f"SELECT diaster.*, osm.geom, osm.lat, osm.lon FROM osm_{scale_dropdown.value} osm INNER JOIN {scale_dropdown.value}_{'flood' if disaster_dropdown.value == 'flood' else 'ls'} diaster ON osm.id = diaster.uid"
    try:
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)

        rows = cursor.fetchall()
        colnames = [desc[0] for desc in cursor.description]

        cursor.close()
        conn.close()
        data = []
        for row in rows:
            geom = wkb.loads(row[colnames.index('geom')], hex=True)
            attributes = {colnames[i]: float(row[i]) if isinstance(row[i], Decimal) else row[i] for i in
                          range(len(row)) if colnames[i] != 'geom'}
            attributes['geometry'] = geom
            data.append(attributes)

        gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')
        m = folium.Map(location=[43.52, 12.29], zoom_start=6, tiles='CartoDB positron')
        # Add basemaps
        folium.TileLayer(name='openstreetmap', show=False).add_to(m)
        folium.TileLayer(name='Stamen Terrain', show=False).add_to(m)
        folium.GeoJson(gdf, name="OSM boundaries").add_to(m)
        # Folium plugins
        MousePosition().add_to(m)
        Draw(export=False).add_to(m)
        MiniMap(position="bottomleft").add_to(m)
        folium.LayerControl().add_to(m)
        # Add markers
        for idx, row in gdf.iterrows():
            popup_content = f'<p style="font-size: 20px;color: darkred; margin-bottom:-10px";font-weight: bolder">{row['nome']}<p/>'
            if disaster_dropdown.value == 'flood':
                for key, value in flood_des.items():
                    if key in colnames and type_dropdown.value in value.lower():
                        popup_content += f"<b>{value}:</b> {row[key]}<br>"
            else:
                for key, value in ls_des.items():
                    if key in colnames and type_dropdown.value in value.lower():
                        popup_content += f"<b>{value}:</b> {row[key]}<br>"
            folium.Marker(
                location=[row.lat, row.lon],
                popup=folium.Popup(popup_content, max_width=450),
                tooltip=row['nome']
            ).add_to(m)
        display(m)
        text.value = ''
    except Exception as error:
        print(f"Error occurs: {error}")


button.on_click(button_clicked)
display(grid, output)

GridspecLayout(children=(Dropdown(description='Scale:', layout=Layout(grid_area='widget001', height='30px', wi…

Output()

In [4]:
Test!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

SyntaxError: invalid syntax (3279454950.py, line 1)

In [5]:
import os
from dotenv import load_dotenv
import psycopg2
from shapely import wkb
import ipywidgets as widgets
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, MousePosition, Draw, MiniMap
import plotly.express as px
from decimal import Decimal
from IPython.display import display, Javascript
import pandas as pd

# Load environment variables
load_dotenv('../backend/database/.env')
db_name = os.getenv('DB_NAME')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')

if not all([db_name, user, password, host]):
    raise ValueError("Please set environment variables in .env file")

# Function to fetch provinces dynamically from the database
def fetch_provinces():
    try:
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute("SELECT DISTINCT nome FROM osm_province ORDER BY nome;")  # Adjust table name if needed
        provinces = [row[0] for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        return provinces
    except Exception as e:
        print(f"Error fetching provinces: {e}")
        return ["Error loading provinces"]

# Add Jupyter Widgets
scale_dropdown = widgets.Dropdown(
    options=['regioni', 'province'],
    description='Scale:',
    layout=widgets.Layout(width='auto', height='30px')
)
disaster_dropdown = widgets.Dropdown(
    options=['flood', 'landslide'],
    description='Disaster:',
    layout=widgets.Layout(width='auto', height='30px')
)
type_dropdown = widgets.Dropdown(
    options=['surface area', 'population', 'families', 'buildings', 'local business', 'cultural heritage'],
    description='Type:',
    layout=widgets.Layout(width='auto', height='30px')
)
province_dropdown = widgets.Dropdown(
    options=fetch_provinces(),  # Fetch provinces dynamically
    description='Province:',
    layout=widgets.Layout(width='auto', height='30px')
)
button = widgets.Button(
    description="Search",
    layout=widgets.Layout(width='80%', height='auto')
)

grid = widgets.GridspecLayout(4, 2)
grid[0, 0] = scale_dropdown
grid[1, 0] = disaster_dropdown
grid[2, 0] = type_dropdown
grid[3, 0] = province_dropdown
grid[:, 1] = button

output = widgets.Output()
text = widgets.Label(value="")

# Function to update chart dynamically based on zoom level
def update_chart(zoom_level):
    data = {
        'Location': ['A', 'B', 'C', 'D', 'E'],
        'Value': [10, 20, 30, 40, 50]
    }
    df = pd.DataFrame(data)

    # Filter data based on zoom level
    filtered_data = df[df['Value'] <= zoom_level]  # Example of filtering
    fig = px.bar(filtered_data, x='Location', y='Value', title=f"Zoom Level: {zoom_level}")

    fig.show()

# Function to handle button click and display map
@output.capture(clear_output=True)
def button_clicked(b):
    text.value = "The map is loading, please wait.."
    display(text)

    query = f"""
    SELECT diaster.*, osm.geom, osm.lat, osm.lon 
    FROM osm_{scale_dropdown.value} osm 
    INNER JOIN {scale_dropdown.value}_{'flood' if disaster_dropdown.value == 'flood' else 'ls'} diaster 
    ON osm.id = diaster.uid
    """

    try:
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)

        rows = cursor.fetchall()
        colnames = [desc[0] for desc in cursor.description]

        cursor.close()
        conn.close()
        data = []
        for row in rows:  #deal with the results of SQL search
            geom = wkb.loads(row[colnames.index('geom')], hex=True)
            attributes = {colnames[i]: float(row[i]) if isinstance(row[i], Decimal) else row[i] for i in range(len(row)) if colnames[i] != 'geom'}
            attributes['geometry'] = geom
            data.append(attributes)

        gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')
        m = folium.Map(location=[43.52, 12.29], zoom_start=6, tiles='CartoDB positron')

        # Add MarkerCluster and data to the map
        marker_cluster = MarkerCluster().add_to(m)
        for idx, row in gdf.iterrows():
            folium.Marker(
                location=[row['lat'], row['lon']],
                popup=f"Location: {row['nome']}", 
            ).add_to(marker_cluster)

        # Folium plugins
        MousePosition().add_to(m)
        Draw(export=False).add_to(m)
        MiniMap(position="bottomleft").add_to(m)
        folium.LayerControl().add_to(m)

        # JavaScript to track zoom level and send it to Python
        zoom_js = """
        setTimeout(function() {
            var maps = document.querySelectorAll('.folium-map');
            if (maps.length > 0) {
                var map = maps[maps.length - 1].folium_map;
                map.on('zoomend', function() {
                    var zoomLevel = map.getZoom();
                    var kernel = Jupyter.notebook.kernel;
                    kernel.execute('update_chart(' + zoomLevel + ')');
                });
            }
        }, 1000);
        """

        display(m)
        display(Javascript(zoom_js))

        text.value = ''

    except Exception as error:
        print(f"Error occurs: {error}")

# Function to filter data based on selected filters
def update_filters(change):
    scale = scale_dropdown.value
    disaster = disaster_dropdown.value
    province = province_dropdown.value
    filtered_data = filter_data(scale, disaster, province)
    update_chart(filtered_data)

def filter_data(scale, disaster, province):
    data = {
        'Location': ['A', 'B', 'C', 'D', 'E'],
        'Value': [10, 20, 30, 40, 50]
    }
    return pd.DataFrame(data)

# Bind the filter function to dropdown changes
scale_dropdown.observe(update_filters, names='value')
disaster_dropdown.observe(update_filters, names='value')
province_dropdown.observe(update_filters, names='value')

# Display the interactive widgets
display(grid, output)

# Add button click event
button.on_click(button_clicked)

GridspecLayout(children=(Dropdown(description='Scale:', layout=Layout(grid_area='widget001', height='30px', wi…

Output()

In [6]:
Version 2~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

SyntaxError: invalid syntax (3185066387.py, line 1)

In [ ]:
import os
from dotenv import load_dotenv
import psycopg2
from shapely import wkb
import ipywidgets as widgets
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster, MousePosition, Draw, MiniMap
import plotly.express as px
from decimal import Decimal
from IPython.display import display, Javascript
import pandas as pd

# Load environment variables
load_dotenv('../backend/database/.env')
db_name = os.getenv('DB_NAME')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')

if not all([db_name, user, password, host]):
    raise ValueError("Please set environment variables in .env file")

# Function to fetch provinces dynamically from the database
def fetch_provinces():   #包含多行province的列表
    try:
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute("SELECT DISTINCT nome FROM osm_province ORDER BY nome;")  # Adjust table name if needed
        provinces = [row[0] for row in cursor.fetchall()]
        cursor.close()
        conn.close()
        return provinces
    except Exception as e:
        print(f"Error fetching provinces: {e}")
        return ["Error loading provinces"]

# Add Jupyter Widgets
scale_dropdown = widgets.Dropdown(
    options=['regioni', 'province'],
    description='Scale:',
    layout=widgets.Layout(width='auto', height='30px')
)
disaster_dropdown = widgets.Dropdown(
    options=['flood', 'landslide'],
    description='Disaster:',
    layout=widgets.Layout(width='auto', height='30px')
)
type_dropdown = widgets.Dropdown(
    options=['surface area', 'population', 'families', 'buildings', 'local business', 'cultural heritage'],
    description='Type:',
    layout=widgets.Layout(width='auto', height='30px')
)
province_dropdown = widgets.Dropdown(
    options=fetch_provinces(),  # Fetch provinces dynamically
    description='Province:',
    layout=widgets.Layout(width='auto', height='30px')
)
button = widgets.Button(
    description="Search",
    layout=widgets.Layout(width='80%', height='auto')
)

grid = widgets.GridspecLayout(4, 2)
grid[0, 0] = scale_dropdown
grid[1, 0] = disaster_dropdown
grid[2, 0] = type_dropdown
grid[3, 0] = province_dropdown
grid[:, 1] = button

output = widgets.Output()
text = widgets.Label(value="")

# Function to update chart dynamically based on zoom level
def update_chart(zoom_level):
    data = {
        'Location': ['A', 'B', 'C', 'D', 'E'],
        'Value': [10, 20, 30, 40, 50]
    }
    df = pd.DataFrame(data)

    # Filter data based on zoom level
    filtered_data = df[df['Value'] <= zoom_level]  # Example of filtering
    fig = px.bar(filtered_data, x='Location', y='Value', title=f"Zoom Level: {zoom_level}")

    fig.show()

# Function to handle button click and display map
@output.capture(clear_output=True)
def button_clicked(b):
    text.value = "The map is loading, please wait.."
    display(text)

    # Query to select disaster data based on user selection
    query = f"""
    SELECT diaster.*, osm.geom, osm.lat, osm.lon 
    FROM osm_{scale_dropdown.value} osm 
    INNER JOIN {scale_dropdown.value}_{'flood' if disaster_dropdown.value == 'flood' else 'ls'} diaster 
    ON osm.id = diaster.uid
    """

    try:     #處理例外 (錯誤處理)
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)

        rows = cursor.fetchall()
        colnames = [desc[0] for desc in cursor.description]

        cursor.close()
        conn.close()
        data = []
        for row in rows:
            geom = wkb.loads(row[colnames.index('geom')], hex=True)
            attributes = {colnames[i]: float(row[i]) if isinstance(row[i], Decimal) else row[i] for i in range(len(row)) if colnames[i] != 'geom'}
            attributes['geometry'] = geom
            data.append(attributes)

        gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')
        m = folium.Map(location=[43.52, 12.29], zoom_start=6, tiles='CartoDB positron')  #設地圖中心

        # Add MarkerCluster and data to the map
        marker_cluster = MarkerCluster().add_to(m)  #Auto make a group of markers while zooming
        for idx, row in gdf.iterrows():       #逐行遍歷
            popup_content = f'<p style="font-size: 20px;color: darkred; margin-bottom:-10px; font-weight: bolder">{row["nome"]}<p/>'
            if disaster_dropdown.value == 'flood':   #represent accessing an attribute of flood
                for key, value in flood_des.items():  # Assuming flood_des has necessary descriptions
                    if key in colnames and type_dropdown.value in value.lower():
                        popup_content += f"<b>{value}:</b> {row[key]}<br>"   #跳出視窗
            else:
                for key, value in ls_des.items():  # Assuming ls_des has necessary descriptions
                    if key in colnames and type_dropdown.value in value.lower():
                        popup_content += f"<b>{value}:</b> {row[key]}<br>"

            folium.Marker(
                location=[row['lat'], row['lon']],
                popup=folium.Popup(popup_content, max_width=450),
                tooltip=row['nome']
            ).add_to(marker_cluster)

        # Add Folium plugins
        MousePosition().add_to(m)
        Draw(export=False).add_to(m)
        MiniMap(position="bottomleft").add_to(m)
        folium.LayerControl().add_to(m)

        # JavaScript to track zoom level and send it to Python
        zoom_js = """
        setTimeout(function() {
            var maps = document.querySelectorAll('.folium-map');
            if (maps.length > 0) {
                var map = maps[maps.length - 1].folium_map;
                map.on('zoomend', function() {
                    var zoomLevel = map.getZoom();
                    var kernel = Jupyter.notebook.kernel;
                    kernel.execute('update_chart(' + zoomLevel + ')');   #傳入縮放層級
                });
            }
        }, 1000);
        """

        display(m)
        display(Javascript(zoom_js))

        text.value = ''

    except Exception as error:
        print(f"Error occurs: {error}")

# Function to filter data based on selected filters
def update_filters(change):
    scale = scale_dropdown.value
    disaster = disaster_dropdown.value
    province = province_dropdown.value
    filtered_data = filter_data(scale, disaster, province)
    update_chart(filtered_data)

def filter_data(scale, disaster, province):   #根據傳入的條件篩選數據
    data = {
        'Location': ['A', 'B', 'C', 'D', 'E'],
        'Value': [10, 20, 30, 40, 50]
    }
    return pd.DataFrame(data)

# Bind the filter function to dropdown changes
scale_dropdown.observe(update_filters, names='value')
disaster_dropdown.observe(update_filters, names='value')
province_dropdown.observe(update_filters, names='value')

# Display the interactive widgets
display(grid, output)

# Add button click event
button.on_click(button_clicked)


## Choropleth map

In [ ]:
import psycopg2
from shapely import wkb
import ipywidgets as widgets
import geopandas as gpd
import folium
from folium.plugins import MousePosition, Draw, MiniMap, HeatMap
from decimal import Decimal
from attributes_descriptions import flood_descriptions as flood_des, landslide_descriptions as ls_des

scale_dropdown = widgets.Dropdown(
    options=['regioni', 'province'],
    description='Scale:',
    layout=widgets.Layout(width='auto', height='30px')
)
disaster_dropdown = widgets.Dropdown(
    options=['flood', 'landslide'],
    description='Disaster:',
    layout=widgets.Layout(width='auto', height='30px')
)
type_dropdown = widgets.Dropdown(
    options=['surface area', 'population', 'families', 'buildings', 'local business', 'cultural heritage'],
    description='Type:',
    layout=widgets.Layout(width='auto', height='30px')
)

button_getdata = widgets.Button(
    description="Search data",
    layout=widgets.Layout(width='80%', height='auto')
)

box_data = widgets.TwoByTwoLayout(top_left=scale_dropdown,
                                  bottom_left=type_dropdown,
                                  bottom_right=button_getdata)

data_dropdown = widgets.Dropdown(
    description='Data:',
    layout=widgets.Layout(width='auto', height='30px')
)

color_dropdown = widgets.Dropdown(
    options=['RdYlGn', 'YlOrRd', 'YlGn', 'RdBu', 'YlGnBu', 'BuGn', 'OrRd', 'Greens', 'Blues', 'Reds'],
    description='Colormap:',
    layout=widgets.Layout(width='auto', height='30px')
)

button_creatmap = widgets.Button(
    description="Creat map",
    layout=widgets.Layout(width='80%', height='auto')
)

output = widgets.Output()
output_map = widgets.Output()

text = widgets.Label(
    value="",
)

box_map = widgets.TwoByTwoLayout(top_left=data_dropdown,
                                 bottom_left=color_dropdown,
                                 bottom_right=button_creatmap)
#Globe variables
data_table = ''
osm_table = ''


# Get the filter data
@output.capture(clear_output=True)
def button_getdata_clicked(b):
    global data_table
    global osm_table
    data_table = f"{scale_dropdown.value}_{'flood' if disaster_dropdown.value == 'flood' else 'ls'}"
    osm_table = f"osm_{scale_dropdown.value}"
    query = f"SELECT column_name FROM information_schema.columns WHERE TABLE_NAME = '{data_table}'"
    try:
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        values = []
        for row in rows:
            if disaster_dropdown.value == 'flood':
                for key, value in flood_des.items():
                    if key in row[0]:
                        values.append(value)
            else:
                for key, value in ls_des.items():
                    if key in row[0]:
                        values.append(value)
        data_dropdown.options = values
        print("Search completed!")
        display(box_map, output_map)
    except Exception as error:
        print(f"Error occurs: {error}")


# Creat choropleth map
@output_map.capture(clear_output=True)
def button_creatmap_clicked(b):
    global data_table
    global osm_table
    text.value = "The map is loading, please wait.."
    display(text)
    try:
        # Throw exception if no value
        if not data_dropdown.value:
            raise Exception("Please choose the data you want to visualization.")
        if disaster_dropdown.value == 'flood':
            for key, value in flood_des.items():
                if value == data_dropdown.value:
                    disaster_data = key

        query = f"SELECT diaster.nome, diaster.{disaster_data}, osm.geom, osm.lat, osm.lon FROM {osm_table} osm INNER JOIN {data_table} diaster ON osm.id = diaster.uid"
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        colnames = [desc[0] for desc in cursor.description]

        cursor.close()
        conn.close()

        data = []
        for row in rows:
            geom = wkb.loads(row[colnames.index('geom')], hex=True)
            attributes = {colnames[i]: float(row[i]) if isinstance(row[i], Decimal) else row[i] for i in
                          range(len(row)) if colnames[i] != 'geom'}
            attributes['geometry'] = geom
            data.append(attributes)

        gdf = gpd.GeoDataFrame(data, crs='EPSG:4326')

        m = folium.Map(location=[43.52, 12.29], zoom_start=6, tiles='CartoDB positron')
        folium.TileLayer(name='openstreetmap', show=False).add_to(m)
        folium.TileLayer(name='cartodbdark_matter', show=False).add_to(m)

        MousePosition().add_to(m)
        Draw(export=False).add_to(m)
        MiniMap(position="bottomleft").add_to(m)

        folium.Choropleth(
            geo_data=gdf,
            name='choropleth',
            data=gdf,
            columns=['nome', disaster_data],
            key_on='feature.properties.nome',
            fill_color=str(color_dropdown.value),
            fill_opacity=0.7,
            line_opacity=0.2,
            legend_name='Statistic Value'
        ).add_to(m)
        folium.LayerControl().add_to(m)

        display(m)
        text.value = ''

    except Exception as error:
        print(f"Error occurs: {error}")


button_getdata.on_click(button_getdata_clicked)
button_creatmap.on_click(button_creatmap_clicked)
display(box_data, output)

## Heat map

In [ ]:
import psycopg2
from shapely import wkb
import ipywidgets as widgets
import geopandas as gpd
import folium
from folium.plugins import MousePosition, Draw, MiniMap, HeatMap
from decimal import Decimal
from attributes_descriptions import flood_descriptions as flood_des, landslide_descriptions as ls_des

scale_dropdown = widgets.Dropdown(
    options=['regioni', 'province'],
    description='Scale:',
    layout=widgets.Layout(width='auto', height='30px')
)
disaster_dropdown = widgets.Dropdown(
    options=['flood', 'landslide'],
    description='Disaster:',
    layout=widgets.Layout(width='auto', height='30px')
)

button_getdata = widgets.Button(
    description="Search data",
    layout=widgets.Layout(width='80%', height='auto')
)

box_data = widgets.TwoByTwoLayout(top_left=scale_dropdown,
                                  bottom_left=disaster_dropdown,
                                  bottom_right=button_getdata)

data_dropdown = widgets.Dropdown(
    description='Data:',
    layout=widgets.Layout(width='auto', height='30px')
)

button_creatmap = widgets.Button(
    description="Creat map",
    layout=widgets.Layout(width='80%', height='auto')
)

output = widgets.Output()
output_map = widgets.Output()

text = widgets.Label(
    value="",
)

box_map = widgets.TwoByTwoLayout(top_left=data_dropdown, bottom_right=button_creatmap)
#Globe variables
data_table = ''
osm_table = ''


# Get the filter data
@output.capture(clear_output=True)
def button_getdata_clicked(b):
    global data_table
    global osm_table
    data_table = f"{scale_dropdown.value}_{'flood' if disaster_dropdown.value == 'flood' else 'ls'}"
    osm_table = f"osm_{scale_dropdown.value}"
    query = f"SELECT column_name FROM information_schema.columns WHERE TABLE_NAME = '{data_table}'"
    try:
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        cursor.close()
        conn.close()
        values = []
        for row in rows:
            if disaster_dropdown.value == 'flood':
                for key, value in flood_des.items():
                    if key in row[0]:
                        values.append(value)
            else:
                for key, value in ls_des.items():
                    if key in row[0]:
                        values.append(value)
        data_dropdown.options = values
        print("Search completed!")
        display(box_map, output_map)
    except Exception as error:
        print(f"Error occurs: {error}")


# Creat choropleth map
@output_map.capture(clear_output=True)
def button_creatmap_clicked(b):
    global data_table
    global osm_table
    text.value = "The map is loading, please wait.."
    display(text)
    try:
        # Throw exception if no value
        if not data_dropdown.value:
            raise Exception("Please choose the data you want to visualization.")
        if disaster_dropdown.value == 'flood':
            for key, value in flood_des.items():
                if value == data_dropdown.value:
                    disaster_data = key
        else:
            for key, value in ls_des.items():
                if value == data_dropdown.value:
                    disaster_data = key
        query = f"SELECT diaster.nome, osm.geom, osm.lat, osm.lon, diaster.{disaster_data} FROM {osm_table} osm INNER JOIN {data_table} diaster ON osm.id = diaster.uid"
        conn = psycopg2.connect(dbname=db_name, user=user, password=password, host=host)
        cursor = conn.cursor()
        cursor.execute(query)
        rows = cursor.fetchall()
        colnames = [desc[0] for desc in cursor.description]

        cursor.close()
        conn.close()

        data = []
        for row in rows:
            attributes = [float(row[i]) for i in range(len(row)) if colnames[i] in ['lon', 'lat', disaster_data ]]
            data.append(attributes)
        
        m = folium.Map(location=[43.52, 12.29], zoom_start=6, tiles='CartoDB positron')
        folium.TileLayer(name='openstreetmap', show=False).add_to(m)
        folium.TileLayer(name='cartodbdark_matter', show=False).add_to(m)

        MousePosition().add_to(m)
        Draw(export=False).add_to(m)
        MiniMap(position="bottomleft").add_to(m)
        
        radius = 30
        if scale_dropdown.value == 'regioni':
            radius = 45
        HeatMap(data=data,
                radius=radius,
                blur=20,
                max_zoom=1).add_to(m)
        folium.LayerControl().add_to(m)

        display(m)
        text.value = ''

    except Exception as error:
        print(f"Error occurs: {error}")


button_getdata.on_click(button_getdata_clicked)
button_creatmap.on_click(button_creatmap_clicked)
display(box_data, output)